In [ ]:
!pip -q install nlpaug

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nlpaug.augmenter.word as naw
import torch

ModuleNotFoundError: No module named 'nlpaug'

In [ ]:
# Fetch data
train_data = pd.read_json("https://raw.githubusercontent.com/google-research-datasets/Disfl-QA/main/train.json", orient='index')
val_data = pd.read_json("https://raw.githubusercontent.com/google-research-datasets/Disfl-QA/main/dev.json", orient='index')
test_data = pd.read_json("https://raw.githubusercontent.com/google-research-datasets/Disfl-QA/main/test.json", orient='index')


#clean data
train_data = train_data[(train_data['disfluent'].str.len() >= 25)]
val_data = val_data[(val_data['disfluent'].str.len() >= 25)]
test_data = test_data[(test_data['disfluent'].str.len() >= 25)]


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [ ]:
class DisfluencyDataset:
    def __init__(self, df, col):
        self.df = df
        self.col = col

    def add_noise(self, batch_size=512):
      translator = naw.BackTranslationAug(from_model_name='Helsinki-NLP/opus-mt-en-fr',
                             to_model_name='Helsinki-NLP/opus-mt-fr-en', device=device)
      filler = naw.ContextualWordEmbsAug(action='insert', model_path='bert-base-uncased', device=device)

      self.df = self.df.drop(columns=['disfluent'])


      # Process sentences in batches
      disfluent_sentences = []

      for i in range(0, len(self.df), batch_size):
        batch = self.df[self.col].iloc[i:i+batch_size].tolist()

        # Apply backtranslation and then filler insertion on the batch
        batch_translated = translator.augment(batch)
        batch_filler = filler.augment(batch_translated)

        disfluent_sentences.extend(batch_filler)


        # Create the new 'disfluent' column
      self.df['disfluent'] = disfluent_sentences
      return self.df


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default.

In [ ]:
# Initialize the DisfluencyDataset class with the DataFrame and the 'original' column
disfluency = DisfluencyDataset(train_data, 'original')
train_augmented = disfluency.add_noise(batch_size=256)
train_augmented = pd.concat([train_data, train_augmented])

disfluency = DisfluencyDataset(test_data, 'original')
test_augmented = disfluency.add_noise(batch_size=512)
test_augmented = pd.concat([test_data, test_augmented])

disfluency = DisfluencyDataset(val_data, 'original')
val_augmented = disfluency.add_noise(batch_size=512)
val_augmented = pd.concat([val_data, val_augmented])

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
output_path = '/content/'

train_augmented.to_csv(f'{output_path}train_augmeneted.csv', index=False)
test_augmented.to_csv(f'{output_path}test_augmeneted.csv', index=False)
val_augmented.to_csv(f'{output_path}val_augmeneted.csv', index=False)


In [4]:
train_augmented.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14292 entries, 0 to 14291
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   original   14292 non-null  object
 1   disfluent  14292 non-null  object
dtypes: object(2)
memory usage: 223.4+ KB


In [5]:
test_augmented.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7258 entries, 0 to 7257
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   original   7258 non-null   object
 1   disfluent  7258 non-null   object
dtypes: object(2)
memory usage: 113.5+ KB


In [6]:
val_augmented.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   original   1990 non-null   object
 1   disfluent  1990 non-null   object
dtypes: object(2)
memory usage: 31.2+ KB
